Este código importa bibliotecas para procesamiento de texto, análisis de datos, aprendizaje automático y utilidades de soporte. 

1. **`pandas` y `numpy`**: Para manipulación y análisis de datos numéricos y tabulares.
2. **`re` y `nltk`**: Para procesamiento de texto. `nltk` proporciona herramientas como `stopwords` (palabras comunes que se eliminan) y `WordNetLemmatizer` (para reducir palabras a su forma base).
3. **`scikit-learn`**: Incluye varias herramientas de aprendizaje automático:
   - **`train_test_split`**: Divide los datos en conjuntos de entrenamiento y prueba.
   - **`TfidfVectorizer`**: Convierte texto en una representación numérica basada en la importancia de las palabras.
   - **Modelos (`RandomForestClassifier`, `SVC`, `MultinomialNB`)**: Para tareas de clasificación.
   - **`KMeans`**: Para agrupar datos no etiquetados en clusters.
   - **Métricas (`accuracy_score`, `classification_report`, etc.)**: Para evaluar el rendimiento de los modelos.
4. **`SMOTE`**: Para equilibrar conjuntos de datos desbalanceados generando muestras sintéticas.
5. **Utilidades**:
   - **`Counter`**: Para contar elementos en datos.
   - **`joblib`**: Para guardar y cargar modelos entrenados.

Este conjunto de importaciones prepara el entorno para una serie de tareas de procesamiento de lenguaje natural (NLP) y aprendizaje automático.

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, silhouette_score, classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib
from sklearn.metrics import silhouette_score

Estas dos líneas descargan recursos necesarios de la biblioteca `nltk`:

1. **`nltk.download('stopwords')`**: Descarga la lista de "stopwords" (palabras comunes) que se suelen eliminar durante el procesamiento de texto, como "y", "el", "es", etc.
2. **`nltk.download('wordnet')`**: Descarga la base de datos de WordNet, que se usa para la lematización, permitiendo reducir las palabras a su forma base.

Ambas descargas son fundamentales para tareas de preprocesamiento de texto en NLP.

In [4]:
# Descargar recursos NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/galotravez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/galotravez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Estas líneas configuran los recursos de preprocesamiento de texto:

1. **`stop_words_spanish = stopwords.words('spanish')`**: Extrae una lista de "stopwords" en español, es decir, palabras comunes que no aportan mucho significado y se suelen eliminar durante el análisis de texto.

2. **`lemmatizer = WordNetLemmatizer()`**: Crea una instancia del lematizador `WordNetLemmatizer`, que se utilizará para reducir las palabras a su forma base (por ejemplo, "corriendo" a "correr").

In [6]:
# Stopwords y lematizador
stop_words_spanish = stopwords.words('spanish')
lemmatizer = WordNetLemmatizer()

Esta función realiza un preprocesamiento avanzado del texto para limpieza y lematización:

1. **`texto.lower()`**: Convierte todo el texto a minúsculas para uniformidad.
2. **`re.sub(r'\W', ' ', texto)`**: Reemplaza todos los caracteres especiales (que no sean letras o números) por un espacio.
3. **`re.sub(r'\s+', ' ', texto)`**: Reemplaza los espacios múltiples por un solo espacio para evitar problemas de formato.
4. **Lematización y eliminación de stopwords**:
   - **`texto.split()`**: Divide el texto en palabras individuales.
   - **`lemmatizer.lemmatize(palabra)`**: Aplica lematización a cada palabra.
   - **`if palabra not in stop_words_spanish`**: Filtra las palabras para eliminar las "stopwords".
   - **`' '.join(...)`**: Une las palabras resultantes en una cadena de texto limpia.

La función devuelve el texto procesado, listo para análisis posterior.

In [8]:
# Preprocesamiento avanzado de texto: Lematización y limpieza
def limpiar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r'\W', ' ', texto)  # Eliminar caracteres especiales
    texto = re.sub(r'\s+', ' ', texto)  # Eliminar espacios extras
    texto = ' '.join([lemmatizer.lemmatize(palabra) for palabra in texto.split() if palabra not in stop_words_spanish])
    return texto

Este bloque de código carga y procesa un conjunto de datos, aplicando preprocesamiento de texto y simplificando las etiquetas de clasificación:

1. **Cargar el dataset:**
   
   - Carga un archivo CSV que contiene las charlas TED en español en un DataFrame llamado `ted_talks_data`.

2. **Limpiar el texto:**
   
   - Aplica la función `limpiar_texto` (definida previamente) para limpiar y lematizar el contenido de la columna `transcript`. Los resultados se almacenan en una nueva columna, `transcript_clean`.

3. **Simplificar etiquetas de clasificación:**
   
   - Define una función que procesa las etiquetas de clasificación (`topics`):
     - Extrae las etiquetas, asumiendo que están en formato de lista (`[topic1, topic2, ...]`).
     - Selecciona el primer tema de la lista y lo devuelve como etiqueta simplificada.
     - Si no hay etiquetas, devuelve "Other".

4. **Aplicar la simplificación:**
   
   - Crea una nueva columna, `simplified_topics`, aplicando la función `simplificar_clases` a la columna `topics`.
   - Identifica las 20 etiquetas más comunes (`top_classes`).
   - Ajusta la columna `simplified_topics`, reemplazando las etiquetas menos frecuentes por "Other".

Este código deja el texto limpio y simplifica las etiquetas, preparándolo para el análisis y modelado posterior.

In [10]:
# Cargar el dataset
dataset_path = 'ted_talks_es.csv'  # Cambia esta ruta
ted_talks_data = pd.read_csv(dataset_path)

# Limpiar el texto con lematización
ted_talks_data['transcript_clean'] = ted_talks_data['transcript'].apply(limpiar_texto)

# Simplificar etiquetas de clasificación agrupando clases poco frecuentes
def simplificar_clases(topics):
    topics = topics[1:-1].split(',')
    if len(topics) > 0:
        return topics[0].strip()  # Usar solo el primer tema
    return 'Other'

ted_talks_data['simplified_topics'] = ted_talks_data['topics'].apply(simplificar_clases)
top_classes = ted_talks_data['simplified_topics'].value_counts().index[:20]
ted_talks_data['simplified_topics'] = ted_talks_data['simplified_topics'].apply(lambda x: x if x in top_classes else 'Other')

Este fragmento selecciona las características y etiquetas que se usarán en el proceso de modelado:

1. **Características (`X`):**
   
   - Asigna la columna `transcript_clean` (el texto preprocesado) a `X`. Esta columna contiene el contenido de las charlas TED ya limpio y lematizado, lo cual se utilizará como entrada para los modelos.

2. **Etiquetas (`y`):**
   
   - Asigna la columna `simplified_topics` a `y`. Esta columna contiene las etiquetas simplificadas que serán las clases o categorías objetivo en el proceso de clasificación.

En resumen, aquí se está preparando el conjunto de datos para entrenar un modelo de clasificación.

In [12]:
# Seleccionar características (transcript_clean) y etiquetas simplificadas
X = ted_talks_data['transcript_clean']
y = ted_talks_data['simplified_topics']

Este fragmento de código vectoriza el texto y balancea las clases para preparar los datos de entrenamiento:

1. **Vectorización TF-IDF:**
   
   - **`TfidfVectorizer`**: Convierte el texto preprocesado en una matriz de características utilizando TF-IDF (Term Frequency-Inverse Document Frequency).
     - **`max_df=0.95`**: Ignora las palabras que aparecen en más del 95% de los documentos, ya que son poco informativas.
     - **`min_df=2`**: Incluye solo las palabras que aparecen al menos en 2 documentos, eliminando términos muy raros.
   - **`fit_transform(X)`**: Aprende las características del texto y las transforma en una representación numérica (matriz dispersa).

2. **Balanceo de clases con SMOTE:**
   
   - **`SMOTE`**: Genera muestras sintéticas para las clases minoritarias para equilibrar el conjunto de datos.
   - **`fit_resample(X_tfidf, y)`**: Aplica SMOTE al conjunto de datos transformado (`X_tfidf`) y sus etiquetas (`y`), devolviendo los datos balanceados en `X_resampled` y `y_resampled`.

3. **Ver distribución de clases:**
   
   - Muestra la distribución de clases después de aplicar SMOTE, utilizando `Counter` para contar la frecuencia de cada clase.

Este bloque asegura que las clases estén equilibradas y que el texto esté preparado para ser utilizado en modelos de clasificación.

In [14]:
# Vectorización TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

print(f"Distribución de clases después de SMOTE: {Counter(y_resampled)}")

Distribución de clases después de SMOTE: Counter({'Other': 1790, "'children'": 1790, "'Africa'": 1790, "'architecture'": 1790, "'activism'": 1790, "'entertainment'": 1790, "'creativity'": 1790, "'business'": 1790, "'culture'": 1790, "'brain'": 1790, "'art'": 1790, "'animals'": 1790, "'biology'": 1790, "'design'": 1790, "'communication'": 1790, "'health'": 1790, "'TED Fellows'": 1790, "'TEDx'": 1790, "'Internet'": 1790, "'TED-Ed'": 1790, "'animation'": 1790})


Este fragmento divide los datos balanceados en conjuntos de entrenamiento y prueba:

1. **`train_test_split`**: 
   
   - **`X_resampled, y_resampled`**: Utiliza los datos y etiquetas balanceados después de aplicar SMOTE.
   - **`test_size=0.2`**: Reserva el 20% de los datos para el conjunto de prueba y el 80% restante para el conjunto de entrenamiento.
   - **`random_state=42`**: Fija una semilla para asegurar la reproducibilidad de la división.

Esta división es necesaria para evaluar el rendimiento del modelo, separando los datos para entrenamiento y validación.

In [16]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

Este bloque de código entrena y evalúa tres modelos de clasificación diferentes utilizando los datos de entrenamiento y prueba:

### 1. **Modelo 1: RandomForest**

- Crea un modelo de bosque aleatorio con 200 árboles y una profundidad máxima de 20.
- Ajusta (entrena) el modelo con los datos de entrenamiento (`X_train`, `y_train`).
- Realiza predicciones en los datos de prueba (`X_test`).
- **Evaluación**:

  - Imprime la precisión y el informe de clasificación, que incluye métricas como precisión, recall y F1-score.

### 2. **Modelo 2: SVM (Support Vector Machine)**

- Crea un clasificador de máquina de soporte vectorial con un kernel lineal.
- Ajusta el modelo con los datos de entrenamiento.
- Realiza predicciones en el conjunto de prueba.
- **Evaluación**:


### 3. **Modelo 3: Naive Bayes**

- Crea un clasificador Naive Bayes multinomial, adecuado para problemas de clasificación de texto.
- Ajusta el modelo con los datos de entrenamiento.
- Realiza predicciones en el conjunto de prueba.
- **Evaluación**:


### Resumen:
- Se entrenan tres modelos diferentes (`RandomForest`, `SVM`, `Naive Bayes`).
- Cada modelo se evalúa en el conjunto de prueba utilizando la precisión y el informe de clasificación para medir su rendimiento.

In [18]:
# Modelo de clasificación 1: RandomForest
clf_rf = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
print("RandomForest - Precisión:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Modelo de clasificación 2: SVM
clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test)
print("SVM - Precisión:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

# Modelo de clasificación 3: Naive Bayes
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)
print("Naive Bayes - Precisión:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

RandomForest - Precisión: 0.9622239957435488
                 precision    recall  f1-score   support

       'Africa'       0.97      0.99      0.98       356
     'Internet'       1.00      1.00      1.00       363
  'TED Fellows'       0.98      0.99      0.99       364
       'TED-Ed'       0.69      1.00      0.82       334
         'TEDx'       0.97      0.95      0.96       351
     'activism'       0.98      0.99      0.99       374
      'animals'       0.99      0.99      0.99       366
    'animation'       0.99      1.00      0.99       303
 'architecture'       1.00      1.00      1.00       356
          'art'       0.99      0.97      0.98       353
      'biology'       0.99      1.00      0.99       380
        'brain'       0.98      0.99      0.99       327
     'business'       0.95      0.97      0.96       369
     'children'       0.99      0.99      0.99       374
'communication'       1.00      0.99      1.00       354
   'creativity'       1.00      0.98      

Este fragmento de código aplica la reducción de dimensionalidad y el agrupamiento para explorar la estructura subyacente de los datos:

### 1. **Reducción de dimensionalidad con PCA:**

- **`PCA` (Análisis de Componentes Principales)**: Reduce la dimensionalidad de los datos, manteniendo la mayor cantidad de varianza posible. Aquí se especifica `n_components=50`, lo que significa que se reduce a 50 componentes principales.
- **`X_tfidf.toarray()`**: Convierte los datos TF-IDF a una matriz densa para ser utilizada por PCA.
- **`fit_transform`**: Ajusta el modelo de PCA a los datos y los transforma a una representación de menor dimensión (`X_pca`).

### 2. **Agrupamiento K-Means con reducción de dimensionalidad:**

- **`KMeans`**: Aplica el algoritmo de agrupamiento `K-Means` para dividir los datos en 5 clusters (`n_clusters=5`).
- **`fit`**: Ajusta el modelo de `KMeans` a los datos reducidos dimensionalmente (`X_pca`).

Este proceso ayuda a explorar posibles grupos dentro del conjunto de datos, utilizando primero la reducción de dimensionalidad para simplificar la estructura antes de realizar el agrupamiento.

In [20]:
# Reducción de dimensionalidad con PCA antes del agrupamiento
pca = PCA(n_components=50, random_state=42)
X_pca = pca.fit_transform(X_tfidf.toarray())

# Agrupamiento K-Means con reducción de dimensionalidad
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_pca)

KMeans(n_clusters=5, random_state=42)

Este bloque evalúa la calidad del agrupamiento K-Means y guarda los modelos entrenados para uso futuro:

### 1. **Calcular la puntuación de silueta:**

- **`silhouette_score`**: Calcula la puntuación de silueta, una métrica que evalúa la calidad del agrupamiento. La puntuación oscila entre -1 y 1; cuanto más cercana a 1, mejor es la separación entre los clusters.
- Muestra la puntuación para evaluar qué tan bien se agruparon los datos.

### 2. **Guardar los modelos y el vectorizador TF-IDF:**

- **`joblib.dump`**: Guarda los modelos entrenados y el vectorizador en archivos para reutilizarlos posteriormente. Aquí se guardan:
  - `clf_rf`: El modelo de RandomForest entrenado.
  - `tfidf_vectorizer`: El vectorizador TF-IDF utilizado para convertir el texto.
- Los archivos se almacenan con los nombres `'modelo_randomforest_smote.pkl'` y `'vectorizador_tfidf_avanzado.pkl'`.

### 3. **Confirmación:**

- Muestra un mensaje confirmando que los modelos y el vectorizador se guardaron exitosamente.

In [22]:
# Calcular la puntuación de silueta
silhouette_avg = silhouette_score(X_pca, kmeans.labels_)
print(f"Puntuación de silueta del agrupamiento K-Means: {silhouette_avg}")

# Guardar los modelos y el vectorizador TF-IDF
joblib.dump(clf_rf, 'modelo_randomforest_smote.pkl')
joblib.dump(tfidf_vectorizer, 'vectorizador_tfidf_avanzado.pkl')

print("Modelos y vectorizador guardados correctamente.")


Puntuación de silueta del agrupamiento K-Means: 0.05642900729825051
Modelos y vectorizador guardados correctamente.
